## 1. Carga de Datos

1.1 – Cargar playlists

In [2]:
import json, pandas as pd
from pathlib import Path

# root = Path(__file__).resolve().parents[1]   # carpeta proyecto
root = Path().resolve()
with open(root/'data/challenge_set.json', 'r', encoding='utf-8') as f:
    playlists_raw = json.load(f)

# Normalizar: una fila por playlist
playlists_df = pd.json_normalize(
    playlists_raw,
    meta=['pid', 'name', 'num_tracks', 'num_holdouts', 'num_samples']
)

# Convierte la lista 'tracks' en una sola columna de listas
print(playlists_df.columns.tolist())


['date', 'version', 'playlists', 'name', 'description']


1.2 – Eliminar playlists vacías


In [3]:
playlists = playlists_raw['playlists']  # Extraer la lista de playlists
playlists_df = pd.json_normalize(
    playlists,
    meta=['pid', 'name', 'num_tracks', 'num_holdouts', 'num_samples']
)
playlists_df = playlists_df[playlists_df['num_samples'] > 0].copy()
print(f"Playlists después del filtro: {len(playlists_df)}")

Playlists después del filtro: 9000


1.3 – Cargar features de AcousticBrainz

In [5]:
# 02_load_acoustic_flat_reduce.py
import orjson, pandas as pd
from pathlib import Path

# root = Path(__file__).resolve().parents[1]

# ---------- 1. Leer JSON ----------
with open(root / 'data/acousticbrainz_data_updated_clean.json', 'rb') as f:
    feat_dict = orjson.loads(f.read())          # {track_id: {...}}

# ---------- 2. Configuración ----------
# Categorías cuyo dict "all" queremos desplegar (mantendremos TODAS sus probabilidades)
CAT_WITH_ALL = {
    "genre_dortmund",
    "genre_electronic",
    "genre_rosamerica",
    "genre_tzanetakis",
    "ismir04_rhythm",
}

# Categorías de las que solo queremos *_value y *_prob (nada más)
CATS_KEEP_VAL_PROB = [
    "danceability", "gender",
    "mood_acoustic", "mood_aggressive", "mood_electronic",
    "mood_happy", "mood_party", "mood_relaxed", "mood_sad",
    "timbre", "tonal_atonal", "voice_instrumental",
]

# ---------- 3. Aplanar ----------
rows = []
for track_id, info in feat_dict.items():
    base = {k: v for k, v in info.items() if k != "highlevel"}  # bpm, energy…

    hl = info.get("highlevel", {})
    for cat, cat_dict in hl.items():
        # Siempre guarda value y prob (los filtraremos luego)
        base[f"{cat}_value"] = cat_dict.get("value")
        base[f"{cat}_prob"]  = cat_dict.get("probability")

        # Si está en la lista de "quiero todas las probs", añade dict all
        if cat in CAT_WITH_ALL:
            for subk, p in cat_dict.get("all", {}).items():
                base[f"{cat}_{subk}"] = p

    base["track_id"] = track_id
    rows.append(base)

tracks_feat_df = pd.DataFrame(rows)

# ---------- 4. Reducir columnas ----------
value_cols = [f"{c}_value" for c in CATS_KEEP_VAL_PROB]
prob_cols  = [f"{c}_prob"  for c in CATS_KEEP_VAL_PROB]

other_cols = [
    col for col in tracks_feat_df.columns
    if not (col.endswith("_value") or col.endswith("_prob"))
]

desired_cols = other_cols + value_cols + prob_cols
df_reduced = tracks_feat_df[desired_cols].copy()

# ---------- 5. Optimizar memoria ----------
float_cols = df_reduced.select_dtypes("float64").columns
df_reduced[float_cols] = df_reduced[float_cols].astype("float32")

print("Columnas originales:", len(tracks_feat_df.columns))
print("Columnas tras reducir:", len(df_reduced.columns))
print("Tamaño en memoria (MB):",
      df_reduced.memory_usage(deep=True).sum() / 1e6)

# ---------- 6. Guardar ----------
out = root / 'data' / 'processed'
out.mkdir(exist_ok=True, parents=True)

df_reduced.to_parquet(
    out / 'tracks_feat_flat.parquet',
    engine='pyarrow',
    compression='snappy',
    index=False
)
df_reduced.head()

Columnas originales: 91
Columnas tras reducir: 78
Tamaño en memoria (MB): 65.021376


,mbid,genre_mb,bpm,energy,danceability_ll,loudness,rating_votes,genre_dortmund_alternative,genre_dortmund_blues,genre_dortmund_electronic,...,mood_acoustic_prob,mood_aggressive_prob,mood_electronic_prob,mood_happy_prob,mood_party_prob,mood_relaxed_prob,mood_sad_prob,timbre_prob,tonal_atonal_prob,voice_instrumental_prob
0,302944eb-7cb7-4045-b99c-c440c362718f,None,125.898476,4.910242,1.067135,0.816509,None,0.012931,0.004027,0.958349,...,0.927582,0.660399,0.905014,0.967353,0.660808,0.927280,0.527017,0.999070,0.990423,0.917732
1,466973b8-6f13-4cdd-aed5-1c8cd6b26fb4,None,128.037704,4.296187,0.937844,0.785428,None,0.001149,0.001203,0.991147,...,0.999994,1.000000,0.917834,0.986929,0.999164,0.808827,0.692220,0.597991,0.913942,0.751203
2,4b789101-f2f6-45f9-8a20-16705010a7a0,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,172cc9c9-3725-452e-81fa-b26f7c3c6c31,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0c14d62c-01f2-4b44-ab8d-a65c97405fde,None,99.882507,4.392409,1.130975,0.920415,None,0.027483,0.015242,0.860394,...,0.984700,0.717427,0.715190,0.945679,0.774370,0.579097,0.765254,0.791275,0.986704,0.988464


1.4 – Expandir playlists a nivel canción

In [6]:

import pandas as pd, ast
from pathlib import Path

# root = Path(__file__).resolve().parents[1]
# playlists_df = pd.read_pickle(root/'data/playlists_df.pkl')

# ‘tracks’ es una lista de dicts → explode
playlists_df['tracks'] = playlists_df['tracks'].apply(lambda x: x)  # ya es lista
pl_tracks = playlists_df.explode('tracks', ignore_index=True)

# Cada dict track → columnas planas
tracks_cols = pd.json_normalize(pl_tracks['tracks'])
playlist_track_df = pd.concat(
    [pl_tracks[['pid','name']], tracks_cols], axis=1
)


1.5 – Unir features

In [7]:
# 1️⃣  Extrae el track-id puro de la URI dentro de las playlists
playlist_track_df['track_id'] = (
    playlist_track_df['track_uri']
      .str.split(':')
      .str[-1]             # último fragmento después del segundo ':'
)

# -------------------------------------------
# 2. Features-level  → asegúrate de tener track_id
#    (tras el script de aplanado, la columna YA se llama track_id)
# -------------------------------------------
# Si viniera como 'track_uri', la renombramos:
# tracks_feat_df.rename(columns={'track_uri': 'track_id'}, inplace=True)

# -------------------------------------------
# 3. Merge por track_id
# -------------------------------------------
playlist_track_full = playlist_track_df.merge(
    tracks_feat_df,
    on='track_id',
    how='left',
    validate='m:1'      # opcional: verifica que cada track_id exista máximo una vez en tracks_feat_df
)

# -------------------------------------------
# 4. Chequeo rápido
# -------------------------------------------
print(
    playlist_track_full[['track_uri', 'track_id', 'loudness']].head()
)
print("NaN en loudness:",
      playlist_track_full['loudness'].isna().mean()*100, "%")

                              track_uri                track_id  loudness
0  spotify:track:66U0ASk1VHZsqIkpMjKX3B  66U0ASk1VHZsqIkpMjKX3B  0.816509
1  spotify:track:5MhsZlmKJG6X5kTHkdwC4B  5MhsZlmKJG6X5kTHkdwC4B  0.785428
2  spotify:track:0GZoB8h0kqXn7XFm4Sj06k  0GZoB8h0kqXn7XFm4Sj06k       NaN
3  spotify:track:35kahykNu00FPysz3C2euR  35kahykNu00FPysz3C2euR       NaN
4  spotify:track:3G6hD9B2ZHOsgf4WfNu7X1  3G6hD9B2ZHOsgf4WfNu7X1  0.920415
NaN en loudness: 55.71103202846975 %


— Verificar % valor_faltante para features faltantes; tomar nota para imputación.

In [8]:
import pandas as pd
from pathlib import Path

# df = pd.read_parquet('data/processed/playlist_track_full.parquet')
df = playlist_track_full

# % de valores faltantes por columna
na_pct = df.isna().mean() * 100
na_pct = na_pct.sort_values(ascending=False)

pd.set_option('display.max_columns', None)      # Muestra todas las columnas
pd.set_option('display.width', None)            # Ajusta el ancho de impresión
pd.set_option('display.expand_frame_repr', False)  # No dividir filas en varias líneas


# Muestra las 20 peores columnas
# print(na_pct.head(20))
print(na_pct)

# (Opcional) Guarda el resumen
na_pct.to_csv('reports/missing_value_percentages.csv')

# Guarda el dataframe en parquet
playlist_track_full.to_parquet("data/processed/playlist_track_full.parquet")



genre_mb           100.0
rating_votes       100.0
rating_value       100.0
danceability_hl    100.0
top_genre_hl       100.0
                   ...  
track_id             0.0
album_uri            0.0
duration_ms          0.0
artist_name          0.0
track_uri            0.0
Length: 101, dtype: float64


1.6 - Guardado de clean Dataframes

In [9]:
from pathlib import Path

# ------------------------------------------------------------------
# Asumimos que ya cargaste playlist_track_full  (merge completo)
# ------------------------------------------------------------------
df = playlist_track_full.copy()

# ==============================================================
# 1)  ELIMINAR COLUMNAS 100 % NaN
# ==============================================================
na_pct = df.isna().mean()
cols_drop = na_pct[na_pct == 1.0].index
df_clean = df.drop(columns=cols_drop)

print(f"Columnas eliminadas por 100 % NaN: {len(cols_drop)}")
# ---------------------------------------------------------------

# ==============================================================
# 2)  DataFrame SOLO con canciones cuyo registro está COMPLETO
#     (sin NaN en ningún atributo numérico que nos interesa)
# ==============================================================
# Elegimos las columnas numéricas relevantes (float / int)
num_cols = df_clean.select_dtypes(include=["number"]).columns

# Filtramos filas que NO tengan faltantes en esas columnas
df_complete = df_clean.dropna(subset=num_cols).reset_index(drop=True)

print("Canciones con datos completos:", df_complete.shape)

# ==============================================================
# 3)  DataFrame con IMPUTACIÓN por media ► dentro de la misma playlist
#     + bandera 'was_imputed' 1/0
# ==============================================================
df_imputed = df_clean.copy()

# ---- 3.1  Calculamos medias por playlist y rellenamos ----------
df_imputed[num_cols] = (
    df_imputed
      .groupby("pid")[num_cols]
      .transform(lambda col: col.fillna(col.mean()))
)

# ---- 3.2  Si aún quedan NaN (por ej. playlist sin ningún valor),
#           rellena con la mediana global como fallback
df_imputed[num_cols] = df_imputed[num_cols].fillna(df_imputed[num_cols].median())

# ---- 3.3  Bandera de imputación --------------------------------
was_na = df_clean[num_cols].isna().any(axis=1)   # True si la fila tenía al menos un NaN
df_imputed["was_imputed"] = was_na.astype(int)

print("Filas imputadas:", df_imputed['was_imputed'].sum())

# ==============================================================
# 4)  (Opcional) Guardar resultados
# ==============================================================
out = Path("data/processed")
out.mkdir(parents=True, exist_ok=True)

df_complete.to_parquet(out / "playlist_tracks_complete.parquet", compression="snappy")
df_imputed.to_parquet(out / "playlist_tracks_imputed.parquet",  compression="snappy")

Columnas eliminadas por 100 % NaN: 7
Canciones con datos completos: (124096, 94)
Filas imputadas: 156904


In [10]:
df_complete.head(), df_imputed.head()

(       pid    name  pos       artist_name                             track_uri                             artist_uri            track_name                             album_uri  duration_ms        album_name                track_id                                  mbid         bpm    energy  danceability_ll  loudness danceability_value  danceability_prob gender_value  gender_prob genre_dortmund_value  genre_dortmund_prob  genre_dortmund_alternative  genre_dortmund_blues  genre_dortmund_electronic  genre_dortmund_folkcountry  genre_dortmund_funksoulrnb  genre_dortmund_jazz  genre_dortmund_pop  genre_dortmund_raphiphop  genre_dortmund_rock genre_electronic_value  genre_electronic_prob  genre_electronic_ambient  genre_electronic_dnb  genre_electronic_house  genre_electronic_techno  genre_electronic_trance genre_rosamerica_value  genre_rosamerica_prob  genre_rosamerica_cla  genre_rosamerica_dan  genre_rosamerica_hip  genre_rosamerica_jaz  genre_rosamerica_pop  genre_rosamerica_rhy  genr

## 2. Analisis Exploratorio (EDA)

2.1. Dimensiones y tipos de variables

In [ ]:
# Dimensiones y tipos de variables
print("Shape:", df_complete.shape)
print("\nTipos de variables:")
print(df_complete.dtypes.value_counts())
print("\nPrimeras filas:")
display(df_complete.head())

2.2. Estadísticas descriptivas de variables numéricas


In [ ]:
# Estadísticas descriptivas de variables numéricas
display(df_complete.describe())

2.3. Distribución de longitud de playlists


In [ ]:
# Distribución de longitud de playlists
import matplotlib.pyplot as plt
playlist_lengths = df_complete.groupby('pid').size()
plt.figure(figsize=(8,4))
playlist_lengths.hist(bins=30)
plt.title("Distribución de longitud de playlists")
plt.xlabel("Número de canciones")
plt.ylabel("Cantidad de playlists")
plt.show()
print("Media:", playlist_lengths.mean(), "Máx:", playlist_lengths.max(), "Min:", playlist_lengths.min())

2.4. Análisis de géneros y moods


In [ ]:
# Análisis de géneros y moods
top_genres = df_complete['genre_dortmund_value'].value_counts().head(10)
top_genres.plot(kind='bar')
plt.title("Top 10 géneros (genre_dortmund_value)")
plt.ylabel("Frecuencia")
plt.show()

top_moods = df_complete['mood_party_value'].value_counts().head(10)
top_moods.plot(kind='bar', color='orange')
plt.title("Top 10 moods (mood_party_value)")
plt.ylabel("Frecuencia")
plt.show()

2.5. Correlaciones numéricas


In [ ]:
# Correlaciones numéricas
import seaborn as sns
num_cols = ['bpm', 'energy', 'danceability_ll', 'loudness']  # Ajusta según tus columnas
corr = df_complete[num_cols].corr(method='pearson')
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Matriz de correlación (Pearson)")
plt.show()

2.6. Outliers numéricos


In [ ]:
# Outliers numéricos con boxplots
for col in ['bpm', 'energy', 'danceability_ll', 'loudness']:
    plt.figure()
    sns.boxplot(x=df_complete[col])
    plt.title(f"Boxplot de {col}")
    plt.show()

2.7. Valores faltantes


In [ ]:
# Porcentaje de valores faltantes por columna
na_pct = df_complete.isna().mean() * 100
na_pct = na_pct.sort_values(ascending=False)
display(na_pct)

# Visualización rápida (opcional)
import missingno as msno
msno.matrix(df_complete.sample(1000, random_state=1))  # Muestra 1000 filas aleatorias
plt.show()

2.8. Canciones y artistas más frecuentes


In [ ]:
# Canciones más repetidas
top_tracks = df_complete['track_name'].value_counts().head(10)
print("Canciones más repetidas:")
display(top_tracks)

# Artistas más frecuentes
top_artists = df_complete['artist_name'].value_counts().head(10)
print("Artistas más frecuentes:")
display(top_artists)

2.9. Diversidad de géneros/artistas por playlist


In [ ]:
# Diversidad de géneros/artistas por playlist
diversity = df_complete.groupby('pid').agg({
    'genre_dortmund_value': pd.Series.nunique,
    'artist_name': pd.Series.nunique
}).rename(columns={'genre_dortmund_value': 'n_genres', 'artist_name': 'n_artists'})
display(diversity.describe())

2.10. Pairplot de variables numéricas


In [ ]:
# Pairplot de variables numéricas
sns.pairplot(df_complete[num_cols].dropna().sample(500, random_state=1))  # Muestra 500 filas
plt.suptitle("Pairplot de variables numéricas", y=1.02)
plt.show()